In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_name = "opencsg/OpenCSG-R1-Qwen2.5-Math-7B-B1"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda:1"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
numbers = "[50, 46, 10]"
target = "86"

messages = [
    {
        "role": "system",
         "content": f"在 <think> </think> 标签中展示你的思考过程，并在 <answer> </answer> 标签中返回最终结论，例如 <answer> (1 + 2) / 3 </answer>。在 <think> 标签中逐步思考。",
    },
    {
        "role": "user",
         "content": f"请分解13124132这个数字",
    },
    {
        "role": "assistant",
        "content": "让我们逐步解决这个问题.",  # 结尾使用 `<think>` 促使模型开始思考
    },
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    # continue_final_message=True,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=1024,
    temperature=0.6
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=False)[0]

In [ ]:
print(f"{response}")

In [ ]:
tokenizer